In [2]:
# standard python
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as spsig

# standard python, say geologists
import obspy

# standard python, say reasonable people
import resampy # for resampling
import librosa # for writing wav files

# custom functions that we need
import sys
sys.path.append('./classes/')
import CMMR_class

import datetime # just to check how long the algorithm takes

%matplotlib inline
#%matplotlib notebook # this is cool... but very slow

## General parameters

In [3]:
n_speakers = 4 # number of speakers
print('Working with ' + str(n_speakers) + ' speakers')
sound_output_dir = '../sounds/output/'

# data sampling (here data = event times)
data_dt = 1 # sampling period of the data, in seconds
data_sr = 1/data_dt

# audio sampling
audio_final_sr = 44100 # when resampling and exporting audio
audio_duration = 60 # duration of final audio tracks, in seconds

Working with 4 speakers


## import data

In [4]:
catalog_dir = '../catalogs/'
catalog_name = 'EQ_Eruption_Catalog_4_1_2018_8_30_2018_HVO.csv'

In [5]:
catalog = pd.read_csv(catalog_dir+catalog_name)

## process data

In [5]:
n_events = len(catalog['latitude'])
##### CHANGE THIS
#n_events = 1000
#####
print('We\'re considering ' + str(n_events) + ' events')

ev_lat = np.array(catalog['latitude'])[0:n_events]
ev_long = np.array(catalog['longitude'])[0:n_events]
ev_mag = np.array(catalog['mag'])[0:n_events]
minmag = np.amin(ev_mag); maxmag = np.amax(ev_mag)

ref_pt = np.array([min(ev_long),min(ev_lat)])  # pour déterminer le rapport de distance entre 1 degré de lon et un degré de lat
ev_X = (ev_long - ref_pt[0])*2.*np.pi/360.*6371.*np.sin(2.*np.pi/360.*(90.-ev_lat))
ev_Y = (ev_lat - ref_pt[1])*2.*np.pi/360.*6371.

ev_times = np.zeros(n_events)
for i in range(n_events):
    ev_times[i] = obspy.UTCDateTime(catalog['time'][i]).timestamp
    
# obspy.UTCDateTime('2018-05-20T21:50:07.310Z').timestamp

# set some variables for the track (in the seismic time domain)
data_t_start = obspy.UTCDateTime(ev_times[0]).timestamp
data_t_end = obspy.UTCDateTime(ev_times[-1]).timestamp
data_duration = data_t_end-data_t_start
data_t = np.arange(data_t_start,data_t_end,data_dt)

print('Data time goes from ' + '{:.0f}'.format(data_t_start) + '\n\t\t to ' + '{:.0f}'.format(data_t_end) + '\n        by steps of ' + str(data_dt) + ' secs.')
print('... that is ' + '{:.0f}'.format(data_t_end-data_t_start) + ' seconds')
print('... that is ' + '{:.2f}'.format((data_t_end-data_t_start)/3600) + ' hours')
print('... that is ' + '{:.2f}'.format((data_t_end-data_t_start)/3600/24) + ' day')

print('Our data array has ' + str(len(data_t)) + ' samples')

We're considering 41328 events
Data time goes from 1522565943
		 to 1535668277
        by steps of 1 secs.
... that is 13102334 seconds
... that is 3639.54 hours
... that is 151.65 day
Our data array has 13102334 samples


In [10]:
LATLON = 0 # set it to 1 if you want to place spks & stations according to latitude & longitude, otherwise it's in kms

## define speed factor

In [11]:
speedfactor = data_duration/audio_duration
audio_working_sr = speedfactor*data_sr
print('We\'ll be working with an audio sampling rate of ' + '{:.1f}'.format(audio_working_sr) + ' Hz')

We'll be working with an audio sampling rate of 218372.2 Hz


## place speakers

In [12]:
if LATLON:
    array_center = [(max(ev_long)+min(ev_long))/2,(max(ev_lat)+min(ev_lat))/2]
else:
    array_center = [(max(ev_X)+min(ev_X))/2,(max(ev_Y)+min(ev_Y))/2]
print('Center of speaker array is @' + str(array_center))

Center of speaker array is @[-155.14408255, 19.36258315]


In [13]:
if LATLON:
    radius = np.sqrt((min(ev_long)-array_center[0])**2+(min(ev_lat)-array_center[1])**2) # distance between speakers & center
else:
    radius = np.sqrt((min(ev_X)-array_center[0])**2+(min(ev_Y)-array_center[1])**2) # distance between speakers & center
radius = radius/2
print('Let\'s draw a polygon of radius ' + str(radius))

angle_shift = -np.pi/4 # rotate the array (in rads) around the center
[x_speakers,y_speakers] = CMMR_class.Coordinates_Polygon(array_center,n_speakers,radius,angle_shift)

Let's draw a polygon of radius 0.19552170647158879


## plot events and speakers

In [ ]:
# display params
arrcent_marksize = 5
arrcent_markcol = 'r'
arrcent_marktype = 's'
arrcent_style = 'none'

spk_marksize = 10
spk_markcol = 'b'
spk_marktype = 'v'
spk_style = 'none'
spk_num_offset = 1 # offset for plotting the speaker number
spk_num_size = 20 # font size for speaker number

ev_markcol = 'k'
ev_marktype = '.'
ev_marksize = 3
ev_style = 'none'

#plot!
fig = plt.figure(figsize=(16,12))
#plt.plot(ev_long,ev_lat,color=ev_markcol,marker=ev_marktype,markersize=ev_marksize,linestyle=ev_style)
if LATLON:
    plt.scatter(ev_long,ev_lat,marker=ev_marktype,s=ev_marksize,cmap='inferno',c=ev_times)
else:
    plt.scatter(ev_X,ev_Y,marker=ev_marktype,s=ev_marksize,cmap='inferno',c=ev_times)
cbar = plt.colorbar(fraction=0.02, pad=0.02)
cbar.set_label('date (time stamp)')
plt.plot(array_center[0],array_center[1],color=arrcent_markcol,marker=arrcent_marktype,markersize=arrcent_marksize,linestyle=arrcent_style)
for k in range(n_speakers):
    plt.plot(x_speakers[k],y_speakers[k],color=spk_markcol,marker=spk_marktype,markersize=spk_marksize,linestyle=spk_style)
    plt.text(x_speakers[k]+spk_num_offset,y_speakers[k]+spk_num_offset,str(k+1),color=spk_markcol, fontsize=spk_num_size)
plt.grid()
plt.gca().set_aspect('equal')
if LATLON:
    plt.ylabel('Latitude (°N)'), plt.xlabel('Longitude (°W)')
else:
    plt.ylabel('Y (kms)'), plt.xlabel('X (kms)')
fig.savefig('../output/Kilauea_MAP.png')

# Generate the data track

as a series of ones at the event times (time stamps)

In [ ]:
data_track = np.zeros(len(data_t))
ev_inds = (ev_times-data_t_start)*data_sr # event occurrence times as indices in the data array

for i in range(n_events):
    index = int(ev_inds[i]) # round to nearest integer, so precision in event times = data sampling period
    data_track[index] = 1

CHECK = 0
if CHECK:
    fig, ax = plt.subplots(figsize=(14,7))
    ax.plot(data_t,data_track)
    plt.grid()
    ax.ticklabel_format(useOffset=False, style='plain')
    
    # check if events are at the right place
    for this in ev_times:
        #print('{:.0f}'.format(this),end='\t')
        plt.plot(this,1,'xr',linewidth=1)

# Generate the audio tracks

## generate the audio click

In [ ]:
clicks_dir = '../sounds/base/'

In [ ]:
CHECKSOUND = 0

#CLICK = 1 # just an ideal impulse, i.e. 1 at 1 point only
#CLICK = 2 # Arthur's descending chirp with skew gaussian envelope
CLICK = 2.5 # Arthur's descending chirp with skew gaussian envelope, frequency decreases with magnitude
#CLICK = 3 # Ben's 

if CLICK == 1:
    click_suffix = '_ClickDirac_'
    click = 1
elif CLICK == 2:
    click_suffix = '_ClickDescChirp_'
    
    ## ALL THAT FOLLOWS IS IN AUDIO TIME
    # say a click is a descending chirp
    click_dur = 0.15 # in seconds 
    t_click = np.arange(0,click_dur,1/audio_working_sr)
    fstart = 300 # start frequency
    fend = .5*fstart
    click = spsig.chirp(t_click, f0=fstart, f1=fend, t1=click_dur, method='log')
    print('Selected audio start frequency for chirp is: ' + '{:.2f}'.format(fstart) + ' Hz')
    print('Selected audio end frequency for chirp is: ' + '{:.2f}'.format(fend) + ' Hz')
    #### THEN LINK audio_fstart TO MAGNITUDE!!!!!!!!!!!!!!
    
    # with a skew gaussian envelope
    t_aux = np.linspace(-1,2,len(t_click))
    sigma = 0.65 # spread factor
    alpha = 50 # Skewness factor

    env = 2*CMMR_class.gaussian(0,sigma,t_aux)*CMMR_class.phi(alpha,t_aux) # skew gaussian
    env = env/np.amax(env) # normalize
    env_thresh = 0.005; env = env[env>env_thresh] # remove values too close to 0
    env = np.interp(t_aux, np.linspace(t_aux[0],t_aux[-1],len(env)), env) # resize array
    env = CMMR_class.fade(env, len(env)/100, len(click)/80, 'lin', np.float32) # fade it in & out
    
    # multiply envelope and click
    click = click*env
    # normalize
    click = .99*click/np.amax(click)
    
    if CHECKSOUND:
        plt.figure(figsize=(16,6))
        plt.plot(t_click,click),plt.plot(t_click,env,'r')
        plt.xlabel('audio time (s)'), plt.xlim([t_click[0],t_click[-1]])
    
        librosa.output.write_wav(clicks_dir + 'click_1.wav', resampy.resample(click,audio_working_sr,audio_final_sr), audio_final_sr, norm=False)
        
elif CLICK == 2.5:
    click_suffix = '_ClickDescChirpMag_'

elif CLICK == 3:
    click_suffix = '_ClickBen_'
    click, sr_click = librosa.load(clicks_dir+'click_0.wav')
    click = resampy.resample(click,sr_click,audio_working_sr)
    click_dur = len(click)/audio_working_sr
    t_click = np.arange(0,click_dur,1/audio_working_sr)
    click = .99*click/np.amax(click)
    
    if CHECKSOUND:
        plt.figure(figsize=(16,6))
        plt.plot(t_click,click)
        plt.xlabel('audio time (s)'), plt.xlim([t_click[0],t_click[-1]])

## lower click amplitude because they'll add up to much more than 1 in amplitude, leading to unbearable distortion

In [ ]:
#click = click/max_evs_in_win
if CLICK == 1:
    pass
elif CLICK == 2:
    click = click/22
elif CLICK == 3:
    click = click/22
elif CLICK == 2.5:
    pass

## initialize tracks for audio rendering

In [ ]:
# initialize tracks (as many as speakers)
for k in range(n_speakers):
    exec('track_' + str(k) + ' = np.zeros(len(data_t))')
    # Audio tracks for now have the same length as the data track
    
audio_t = np.arange(0,audio_duration,1/audio_working_sr)

## fill in the audio tracks

In [ ]:
tbefore = datetime.datetime.now()

for i in range(n_events):
#for i in range(1000):
    
    #print('>> event #' + str(i))
    distances = np.zeros(n_speakers) # for current event: distance from each speaker
    sum__ = 0 # store the sum of inverse square distances

    # 1- distance from current speaker
    for k in range(n_speakers):
        if LATLON:
            distances[k] = CMMR_class.EuclDistance([ev_long[i],ev_lat[i]],[x_speakers[k],y_speakers[k]])
        else:
            distances[k] = CMMR_class.EuclDistance([ev_X[i],ev_Y[i]],[x_speakers[k],y_speakers[k]])
        sum__ = sum__+1/(distances[k]**2)

    C = np.sqrt(1/sum__) # normalization constant
    #print('C = ' + str(C))
    #print('distances = ' + str(distances))

    # 2- fill the track with a click at the time of event, amplitude corresponding to ev/speaker distance
    for k in range(n_speakers):
        if CLICK == 1:
            index = int(ev_inds[i])
            exec('track_' + str(k) + '[index]' + ' = click*C/distances[k]')
        elif CLICK == 2:
            start_index = int(ev_inds[i])
            end_index = start_index + len(click)
            if end_index < len(data_t):
                exec('track_' + str(k) + '[start_index:end_index]' + ' = track_' + str(k) + '[start_index:end_index] + click*C/distances[k]')
                #plt.plot(click*C/distances[k]+k)
                #eval('plt.plot(track_' + str(k) + ')')
        elif CLICK == 3:
            start_index = int(ev_inds[i])
            end_index = start_index + len(click)
            if end_index < len(data_t):
                exec('track_' + str(k) + '[start_index:end_index]' + ' = track_' + str(k) + '[start_index:end_index] + click*C/distances[k]')
        elif CLICK == 2.5:
            fmin = 120
            fmax = 600

            click_dur = 0.2 # in seconds 
            t_click = np.arange(0,click_dur,1/audio_working_sr)

            fstart = CMMR_class.linmap(ev_mag[i],minmag,maxmag,fmax,fmin)
            fend = .5*fstart
            click = spsig.chirp(t_click, f0=fstart, f1=fend, t1=click_dur, method='log')

            t_aux = np.linspace(-1,2,len(t_click))
            sigma = 0.65 # spread factor
            alpha = 50 # Skewness factor

            env = 2*CMMR_class.gaussian(0,sigma,t_aux)*CMMR_class.phi(alpha,t_aux) # skew gaussian
            env = env/np.amax(env) # normalize
            env_thresh = 0.005; env = env[env>env_thresh] # remove values too close to 0
            env = np.interp(t_aux, np.linspace(t_aux[0],t_aux[-1],len(env)), env) # resize array
            env = CMMR_class.fade(env, len(env)/100, len(click)/80, 'lin', np.float32) # fade it in & out

            # multiply envelope and click
            click = click*env
            # normalize
            click = .99*click/np.amax(click)
            click = click/40
        
            start_index = int(ev_inds[i])
            end_index = start_index + len(click)
            if end_index < len(data_t):
                exec('track_' + str(k) + '[start_index:end_index]' + ' = track_' + str(k) + '[start_index:end_index] + click*C/distances[k]')
                
tafter = datetime.datetime.now()

print('Elapsed time: ' + str(tafter - tbefore))

## Plot the track(s)

In [ ]:
CHECKTRACK = 0
n_track_to_check = 0
exec('track_to_check = track_' + str(n_track_to_check))

if CHECKTRACK:
    fig, ax = plt.subplots(2,figsize=(14,7))
    ax[0].plot(data_t,data_track,'k')
    #plt.xlim([data_t[0],data_t[-1]])
    ax[1].plot(audio_t,track_to_check,'k')
    plt.xlim([audio_t[0],audio_t[-1]])
    #plt.xlim([ev_times[0],ev_times[3020]])
    #plt.xlim([ev_times[0],ev_times[10]])
    plt.grid()
    ax[1].ticklabel_format(useOffset=False, style='plain')

# Audify!!

In [ ]:
# resample to audio rate & fade in/out
for k in range(n_speakers):
    exec('track_' + str(k) + ' = resampy.resample(track_' + str(k) + ', audio_working_sr, audio_final_sr)') # resample
    exec('track_' + str(k) + ' = CMMR_class.fade(track_' + str(k) + ', 0.3*audio_final_sr,0.3*audio_final_sr,\'lin\',np.float32)') # fade in & out
    
audio_final_t = resampy.resample(audio_t,audio_working_sr,audio_final_sr)

In [ ]:
# normalize --> BAD IDEA FOR NOW, constant C already prevents amplitudes from exceeding 1
#for k in range(n_speakers):
#    exec('track_' + str(k) + '=.9*track_' + str(k) + '/np.amax(np.abs(track_' + str(k) + '))')

In [ ]:
# plot if you dare
PLOT = 1

if PLOT: 
    plt.figure(figsize=(10,20))
    for k in range(n_speakers):
        plt.subplot(n_speakers,1,k+1)
        eval('plt.plot(audio_final_t,track_' + str(k) + ',\'k\')')
        #eval('plt.plot(track_' + str(k) + ',\'k\')')
        plt.grid()
        plt.title('Speaker number ' + str(k+1))
        plt.ylim([-1,1])
        plt.xlim([audio_final_t[0],audio_final_t[-1]])
    plt.tight_layout()
    plt.xlabel('audio time (s)')

In [ ]:
base_name = 'Kilauea_K1c_' + str(n_speakers) + 'spks' + click_suffix
for k in range(n_speakers):
    eval('librosa.output.write_wav(sound_output_dir + base_name + ' + '\'chan' + str(k+1) + '.wav\'' + ', track_' + str(k) + ', ' + 'audio_final_sr' + ', norm=False)')